In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from matplotlib import cm

In [2]:
# 让图片中可以显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'
# 让图片中可以显示负号
plt.rcParams['axes.unicode_minus'] = False

In [3]:
file_path = "/home/shuai/视频/old20240324.csv"

In [4]:
import os
import shutil

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        if os.path.splitext(folder_path)[1].lower() != '.zip':
            os.makedirs(folder_path)
            print(f"文件夹 {folder_path} 创建成功")
    else:
        if os.path.isfile(folder_path):
            os.remove(folder_path)
            print(f"文件 {folder_path} 已删除")
        elif os.path.isdir(folder_path):
            shutil.rmtree(folder_path)
            print(f"文件夹 {folder_path} 及其内容已删除")
            os.makedirs(folder_path)
            print(f"文件夹 {folder_path} 创建成功")

# 提取文件名称和路径
file_name = os.path.splitext(os.path.basename(file_path))[0]
directory_path = os.path.dirname(file_path)
# 你可以调用这个函数，并传入需要检查的文件夹路径
folder_path = directory_path + "/" + file_name
zip_path = directory_path + "/" + file_name + ".zip"
create_folder_if_not_exists(folder_path)
create_folder_if_not_exists(zip_path)


文件夹 /home/shuai/视频/old20240324 及其内容已删除
文件夹 /home/shuai/视频/old20240324 创建成功
文件 /home/shuai/视频/old20240324.zip 已删除


In [5]:
# 读取Excel文件，忽略前6行
df0 = pd.read_csv(file_path)
display(df0.head())
display(df0.shape)
df0_rows = int(df0.shape[0]/9)
df0_cols = int(df0.shape[1]-2)
df0_values = df0.values

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35
0,1.5,0.5144,13768.68066,286696.8125,28747.72070,354003.7188,34314.93359,357846.5000,56136.91797,395813.1875,...,96071.80469,368650.3438,59487.93359,296156.0000,8303.420898,159177.4844,71481.86719,3.669561e+05,0.000000,0.000000
1,1.5,1.0290,15661.30762,239330.3906,30706.92188,278924.6875,35697.22656,279872.7500,56019.22266,327893.5000,...,96441.28125,363170.9375,58987.83203,288542.5625,5880.359375,130922.3516,62098.50391,3.350240e+05,0.221023,0.334890
2,1.5,1.5430,14526.57227,227596.4688,27759.19727,266642.8125,32271.85352,275117.9375,51267.17188,325726.9375,...,97583.65625,351476.6875,58434.10156,289191.4063,5149.017090,136740.6406,59760.77734,3.146491e+05,0.214608,0.315642
3,2.0,0.5144,36442.47656,552685.3750,57913.85547,754881.6875,64295.28906,780469.2500,93842.42969,892707.5000,...,129560.60940,842952.8750,87147.53125,708814.8750,50588.695310,621670.2500,153912.14060,1.021330e+06,1.127920,1.104291
4,2.0,1.0290,35045.92188,498700.3438,58710.47656,656219.5625,64407.51953,671388.8750,90042.95313,754463.6875,...,129916.44530,662650.7500,87981.36719,588829.7500,40036.625000,477864.0313,101340.08590,7.536669e+05,0.538425,0.781815


(45, 36)

In [6]:
# 定义插值函数
def interpolate_surface(x, y, z):
    # 创建网格点
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    
    # 进行插值
    zi = griddata((x, y), z, (xi, yi), method='cubic')
    return xi, yi, zi

In [7]:
# 循环处理每个子图
for j in range(df0_cols):
    for i in range(df0_rows):
        x = df0_values[i*9:(i+1)*9, 1].reshape(-1)
        y = df0_values[i*9:(i+1)*9, 0].reshape(-1)
        if j < 32: 
            z = df0_values[i*9:(i+1)*9, j+2].reshape(-1) / 10000
        else:
            z = df0_values[i*9:(i+1)*9, j+2].reshape(-1)
        
        # 插值得到曲面
        xi, yi, zi = interpolate_surface(x, y, z)
        
        # 创建一个三维坐标系
        fig = plt.figure(figsize=(140/25.4,140/25.4))
        ax = fig.add_subplot(111, projection='3d')
        # 绘制曲面
        ax.plot_surface(xi, yi, zi, cmap=cm.coolwarm)
        if np.any(zi >=49): 
            ax.contourf(xi, yi, zi, levels=[49.9,50.1], colors='black')
        if np.any(zi >=195): 
            ax.contourf(xi, yi, zi, levels=[195.9,196.1], colors='green')
        # 绘制散点图
        ax.scatter(x, y, z)

        file_save_path = ""
        if j < 32: 
            jun_zui = '均值' if j%2==0 else '最值'
            ax.set_xlabel('流速(m/s)', fontsize=9)
            ax.set_ylabel('波高(m)', fontsize=9)
            ax.set_zlabel('缆绳张力(t)', fontsize=9)
            plt.title(f'{i*45}度缆绳{int(j/2)+1}张力{jun_zui}')
            file_save_path = f'{folder_path}/{i*45}度缆绳{int(j/2)+1}张力{jun_zui}.jpg'
        else:
            heng_zong = '横荡' if j%2==0 else '纵荡'
            ax.set_xlabel('流速(m/s)', fontsize=9)
            ax.set_ylabel('波高(m)', fontsize=9)
            ax.set_zlabel(f'{heng_zong}(m)', fontsize=9)
            plt.title(f'{i*45}度船舶{heng_zong}运动幅值')
            file_save_path = f'{folder_path}/{i*45}度船舶{heng_zong}运动幅值.jpg'
        # 设置边缘空白大小    
        plt.subplots_adjust(left=0.05, right=0.85, top=0.9, bottom=0)
        plt.savefig(file_save_path)
        # 关闭图形，以便下一次循环创建新的图形
        plt.close()

In [8]:
import zipfile

def zip_folder(folder_path, zip_file_name):
    with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# 定义要压缩的文件夹路径和ZIP文件名
zip_file_name = folder_path + ".zip"

# 调用函数压缩文件夹
zip_folder(folder_path, zip_file_name)